# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratório 1

Neste laboratório iremos realizar a conexão com o Azure OpenAI e executar diversas tarefas: solicitar respostas da API, usar respostas baseadas em texto, analisar as respostas obtidas, realizar a conversão de texto em embeddings, fazer chamadas à API enviando imagens e também realizar chamadas a outros modelos LLM.

O primeiro passo é a validação da configuração das variáveis de ambiente no arquivo `.env` presente na raiz do repositório.

Preencha os valores das variáveis de acordo com o solicitado.

### Exercício 1 - Chamada à API

Vamos realizar a importação das bibliotecas necessárias para o laboratório.

In [ ]:
#%pip install -r ../../requirements.txt

In [ ]:
#%pip install openai dotenv

In [ ]:
import json
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../../.env")

Vamos carregar as credenciais em variáveis para facilitar o uso no laboratório.

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
api_version=os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

Agora vamos iniciar o client com as credenciais fornecidas.

In [ ]:
client = AzureOpenAI(
  azure_endpoint = azure_endpoint[0], 
  api_key=api_key[0],  
  api_version=api_version
)


Após criarmos o cliente, vamos realizar uma chamada simples onde passaremos:

1. Uma mensagem para a role "system" definindo o papel da LLM
2. Uma pergunta inicial do usuário
3. Uma resposta do assistente demonstrando como ele deve responder (exemplo)
4. Uma nova pergunta para ele responder baseado no contexto estabelecido anteriormente

In [ ]:
response = client.chat.completions.create(
    model=deployment_name, 
    messages=[
        {"role": "system", "content": "Você é um assistente útil."},
        {"role": "user", "content": "O Azure OpenAI suporta chaves gerenciadas pelo cliente?"},
        {"role": "assistant", "content": "Sim, chaves gerenciadas pelo cliente são suportadas pelo Azure OpenAI."},
        {"role": "user", "content": "Outros serviços do Azure também suportam isso?"}
    ]
)

Agora vamos acessar diretamente a resposta da LLM.

In [ ]:
print(response.choices[0].message.content)

### Exercício 2 - Analisando a Resposta

Agora que fizemos uma chamada para o Azure OpenAI, vamos analisar o conteúdo completo da resposta:

In [ ]:
response

Agora vamos estruturar a resposta em um formato mais legível para melhor visualização dos dados:

In [ ]:
response_dict = {
    "id": response.id,
    "model": response.model,
    "created": response.created,
    "usage": {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    },
    "completion_tokens_details": {
        "accepted_prediction_tokens": response.usage.completion_tokens_details.accepted_prediction_tokens,
        "audio_tokens": response.usage.completion_tokens_details.audio_tokens,
        "reasoning_tokens": response.usage.completion_tokens_details.reasoning_tokens,
        "rejected_prediction_tokens": response.usage.completion_tokens_details.rejected_prediction_tokens
    },
    "choices": [{
        "index": choice.index,
        "message": {
            "role": choice.message.role,
            "content": choice.message.content
        },
        "finish_reason": choice.finish_reason,
        "content_filter_results": choice.content_filter_results
    } for choice in response.choices],
    "prompt_filter_results": response.prompt_filter_results
}

print(json.dumps(response_dict, indent=2, ensure_ascii=False))

A API não responde apenas com o texto gerado pela LLM. Temos muito mais informações nessa resposta, como por exemplo:
- Se usa áudio ou imagem
- Filtragem de conteúdo
- Avaliação de conteúdo
- Contagem de tokens do prompt
- Contagem de tokens gerados na resposta
- Detalhes sobre tokens de raciocínio (para modelos que suportam)
- Resultados de filtros aplicados

Essas informações são essenciais para monitoramento, custos e controle de qualidade da aplicação.

Após realizar a chamada e explorar a resposta, teste você também criando um prompt personalizado. Realize experimentos com os seguintes parâmetros importantes:

- **max_completion_tokens**: Número máximo de tokens que podem ser gerados na resposta
- **temperature**: Controla a criatividade (0.0 = mais determinístico, 1.0 = mais criativo)
- **top_p**: Controla a diversidade da resposta via nucleus sampling
- **frequency_penalty**: Penaliza repetição de tokens baseado na frequência
- **presence_penalty**: Penaliza repetição de tokens independentemente da frequência

In [ ]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Você é um assistente útil.",
        },
        {
            "role": "user",
            "content": "Vou viajar para Paris, o que devo ver?",
        },
        {
            "role": "assistant",
            "content": "Paris, a capital da França, é conhecida por sua arquitetura deslumbrante, museus de arte, marcos históricos e atmosfera romântica. Aqui estão algumas das principais atrações para ver em Paris:\n \n 1. A Torre Eiffel: A icônica Torre Eiffel é um dos marcos mais reconhecíveis do mundo e oferece vistas deslumbrantes da cidade.\n 2. O Museu do Louvre: O Louvre é um dos maiores e mais famosos museus do mundo, abrigando uma impressionante coleção de arte e artefatos, incluindo a Mona Lisa.\n 3. Catedral de Notre-Dame: Esta bela catedral é um dos marcos mais famosos de Paris e é conhecida por sua arquitetura gótica e vitrais deslumbrantes.\n \n Estas são apenas algumas das muitas atrações que Paris tem a oferecer. Com tanto para ver e fazer, não é de admirar que Paris seja um dos destinos turísticos mais populares do mundo.",
        },
        {
            "role": "user",
            "content": "O que há de tão especial no #1?",
        }
    ],
    max_completion_tokens=800,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model=deployment_name
)

print(response.choices[0].message.content)

### Exercício 3 - Embeddings

Os embeddings são representações numéricas de texto que capturam o significado semântico das palavras ou frases. No Azure OpenAI, você pode usar o modelo de embeddings para converter texto em vetores numéricos que podem ser usados para tarefas como busca semântica, classificação e análise de similaridade.

Para mais informações sobre como trabalhar com embeddings no Azure OpenAI, consulte a [documentação oficial](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings?tabs=python-new).

In [ ]:
response = client.embeddings.create(
    input = "cachorro",
    model= embedding_model
)

print(response.model_dump_json(indent=2))

Aqui geramos o embedding de uma única palavra, mas podemos fazer o mesmo para trechos de texto maiores. O modelo organizará automaticamente o conteúdo em vetores numéricos que capturam o significado semântico.

Para armazenar embeddings podemos usar uma série de serviços disponíveis no Azure. Basta escolher o que mais se adequa à sua solução:

- [Azure AI Search](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)
- [Azure Cosmos DB for MongoDB vCore](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search)
- [Azure SQL Database](https://learn.microsoft.com/en-us/azure/azure-sql/database/ai-artificial-intelligence-intelligent-applications?view=azuresql&preserve-view=true#vector-search)
- [Azure Cosmos DB for NoSQL](https://learn.microsoft.com/en-us/azure/cosmos-db/vector-search)
- [Azure Cosmos DB for PostgreSQL](https://learn.microsoft.com/en-us/azure/cosmos-db/postgresql/howto-use-pgvector)
- [Azure Database for PostgreSQL - Flexible Server](https://learn.microsoft.com/en-us/azure/postgresql/flexible-server/how-to-use-pgvector)
- [Azure Cache for Redis](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/cache-tutorial-vector-similarity)
- [Use Eventhouse as a vector database - Real-Time Intelligence in Microsoft Fabric](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/vector-database)

### Exercício 4 - Processamento de Imagens

No Azure AI Foundry podemos trabalhar com modelos que processam imagens, tanto para geração de imagens quanto modelos multimodais nos quais podemos usar imagens como contexto. Neste exercício vamos aprender como utilizar imagens como contexto do prompt.

**Primeiro ponto importante**: temos que pensar em como enviar uma imagem junto ao prompt. Para isso temos 2 opções principais:
1. Enviar a imagem junto com o prompt via base64 (codificada)
2. Enviar a imagem como um link/URL

Vamos ver os 2 exemplos práticos a seguir.

Primeiro, vamos aproveitar o cliente que já instanciamos e enviar uma URL de uma imagem, pedindo para o modelo descrevê-la:

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Itaim_Bibi_Business_District.jpg/250px-Itaim_Bibi_Business_District.jpg"


<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Itaim_Bibi_Business_District.jpg/250px-Itaim_Bibi_Business_District.jpg" alt="Azure AI Foundry" width="600">

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "Você é um assistente útil." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Descreva essa imagem:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": image_url
                }
            }
        ] } 
    ],
    max_tokens=2000 
)
print(response.choices[0].message.content)

Agora vamos ler uma imagem local armazenada em nosso sistema e enviá-la junto com a mensagem:

In [ ]:
import base64
from mimetypes import guess_type

In [ ]:
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

In [ ]:
image_path = "../../../samples/234039841.jpg"
data_url = local_image_to_data_url(image_path)
print("Data URL:", data_url)

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "Você é um assistente útil." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Descreva essa imagem:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }
        ] } 
    ],
    max_tokens=2000 
)
print(response.choices[0].message.content)

Utilizando o Azure OpenAI temos acesso a diversos tipos de funcionalidades além das que exploramos aqui. Recomendo navegar e explorar as opções disponíveis para entender qual é a melhor abordagem para sua aplicação específica:

- [Responses API](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/responses)
- [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning)
- [Chat completions API](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chatgpt)
- [Computer Use](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/computer-use)
- [Model router concepts](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/model-router)
- [Function calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling)
- [Predicted outputs](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/predicted-outputs)
- [Prompt caching](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/prompt-caching)
- [Structured outputs](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs)
- [Vision-enabled chats](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs)
- [JSON Mode](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/json-mode)
- [Reproducible output](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reproducible-output)

### Exercício 5 - Outros modelos no Azure AI Foundry

Através do Azure AI Foundry podemos explorar uma série de modelos disponíveis no [Model Catalog](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/foundry-models-overview). 

Lá temos acesso a modelos que são disponibilizados pela Microsoft (OpenAI, Meta, Mistral AI, Deepseek, xAI, Black Forest Labs) bem como modelos disponibilizados por parceiros e pela comunidade (Nixtla, AI21, NTT Data, Core42, NVIDIA NIM Microservices, Stability AI). 

Através da documentação fornecida é possível entender a diferença entre os diferentes modos de disponibilização dos modelos e como escolher de acordo com seu cenário específico.



Agora vamos seguir com um exemplo prático de como chamar um modelo disponibilizado pelo Azure AI Foundry através de uma chamada de chat completion usando uma biblioteca diferente da anterior:

In [ ]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage

In [ ]:
%pip install azure-ai-inference azure-core

In [ ]:
endpoint = os.getenv("AZURE_PHI4_ENDPOINT")
api_key = os.getenv("AZURE_PHI4_API_KEY")
model_name = os.getenv("AZURE_PHI4_DEPLOYMENT")


In [ ]:
clientPhi = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key),
    api_version="2024-05-01-preview"
)

In [ ]:
response = clientPhi.complete(
    messages=[
        SystemMessage(content="Você é um assistente útil."),
        UserMessage(content="Vou viajar para Paris, o que devo ver?"),
    ],
    max_tokens=2048,
    temperature=0.8,
    top_p=0.1,
    presence_penalty=0.0,
    frequency_penalty=0.0,
    model=model_name
)

print(response.choices[0].message.content)

## 🎯 Atividades Práticas 

Agora que você explorou os conceitos básicos do Azure AI Foundry, vamos praticar com algumas atividades simples e direcionadas para consolidar o aprendizado!

### 📝 Atividade 1: Teste de Temperatura
**Objetivo**: Entender como a temperatura afeta a criatividade das respostas.

Execute o código abaixo e observe como o mesmo prompt gera respostas diferentes com temperaturas variadas:

In [ ]:
prompt = "Escreva um slogan criativo para uma empresa de tecnologia."

# Testando diferentes temperaturas
temperaturas = [0.1, 0.5, 1.0]

for temp in temperaturas:
    print(f"\n🌡️ TEMPERATURA: {temp}")
    print("-" * 40)
    
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Você é um assistente criativo de marketing."},
            {"role": "user", "content": prompt}
        ],
        temperature=temp,
        max_completion_tokens=100
    )
    
    print(response.choices[0].message.content)
    print(f"Tokens usados: {response.usage.total_tokens}")

### 🔍 Atividade 2: Comparação de Embeddings
**Objetivo**: Comparar como palavras similares têm embeddings próximos.

Vamos gerar embeddings para palavras relacionadas e ver seus tamanhos:

In [ ]:
%pip install numpy

In [ ]:
import numpy as np

# Palavras para comparar
palavras = ["gato", "felino", "cachorro", "cão", "automóvel", "carro"]

embeddings_dict = {}

print("Gerando embeddings para as palavras...")
for palavra in palavras:
    response = client.embeddings.create(
        input=palavra,
        model=embedding_model
    )
    embedding = response.data[0].embedding
    embeddings_dict[palavra] = embedding
    print(f"✅ {palavra}: {len(embedding)} dimensões")

print(f"\nPrimeiros 5 valores do embedding da palavra 'gato':")
print(embeddings_dict["gato"][:5])

In [ ]:
# Função para calcular similaridade de cosseno
def calcular_similaridade(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Comparando similaridades
print("🔍 Comparando similaridades:")
print("-" * 50)

# Gato vs Felino
sim_gato_felino = calcular_similaridade(embeddings_dict["gato"], embeddings_dict["felino"])
print(f"Gato ↔ Felino: {sim_gato_felino:.3f}")

# Cachorro vs Cão
sim_cachorro_cao = calcular_similaridade(embeddings_dict["cachorro"], embeddings_dict["cão"])
print(f"Cachorro ↔ Cão: {sim_cachorro_cao:.3f}")

# Automóvel vs Carro
sim_auto_carro = calcular_similaridade(embeddings_dict["automóvel"], embeddings_dict["carro"])
print(f"Automóvel ↔ Carro: {sim_auto_carro:.3f}")

# Gato vs Carro (deve ser baixa)
sim_gato_carro = calcular_similaridade(embeddings_dict["gato"], embeddings_dict["carro"])
print(f"Gato ↔ Carro: {sim_gato_carro:.3f}")

print(f"\n💡 Palavras similares têm similaridade mais alta (próxima de 1.0)!")

### 🖼️ Atividade 3: Análise de Imagem com Diferentes Prompts
**Objetivo**: Testar como diferentes prompts afetam a análise da mesma imagem.

Vamos usar diferentes tipos de perguntas para a mesma imagem:

In [ ]:
# Usando a mesma imagem com diferentes prompts
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Itaim_Bibi_Business_District.jpg/250px-Itaim_Bibi_Business_District.jpg"

# Diferentes tipos de análise
prompts = [
    "Descreva esta imagem em uma frase:",
    "Que tipo de local é este?",
    "Quais cores predominam nesta imagem?",
    "Esta imagem transmite que sensação?",
    "Conte os prédios que você consegue ver:"
]

for i, prompt_text in enumerate(prompts, 1):
    print(f"\n🔍 PERGUNTA {i}: {prompt_text}")
    print("-" * 60)
    
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Você é um assistente especializado em análise de imagens."},
            {"role": "user", "content": [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": image_url}}
            ]}
        ],
        max_tokens=150
    )
    
    print(response.choices[0].message.content)

### 🔢 Atividade 4: Contador de Tokens
**Objetivo**: Entender como o tamanho do prompt afeta o consumo de tokens.

Vamos testar prompts de diferentes tamanhos e ver o impacto nos tokens:

In [ ]:
# Prompts de diferentes tamanhos
prompts_teste = [
    "Olá",
    "Explique o que é inteligência artificial",
    "Explique detalhadamente o que é inteligência artificial, como funciona, suas aplicações práticas, benefícios e desafios para a sociedade moderna"
]

print("📊 ANÁLISE DE CONSUMO DE TOKENS")
print("=" * 50)

for i, prompt in enumerate(prompts_teste, 1):
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Você é um assistente útil."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=100  # Limitando resposta para focar no prompt
    )
    
    print(f"\n🔍 TESTE {i}:")
    print(f"Prompt: '{prompt[:50]}{'...' if len(prompt) > 50 else ''}'")
    print(f"Tokens do prompt: {response.usage.prompt_tokens}")
    print(f"Tokens da resposta: {response.usage.completion_tokens}")
    print(f"Total de tokens: {response.usage.total_tokens}")
    print(f"Resposta: {response.choices[0].message.content[:100]}...")

print("\n💡 Prompts maiores consomem mais tokens de entrada!")

### 🎭 Atividade 5: Teste de Personas
**Objetivo**: Ver como diferentes personas (system messages) afetam as respostas.

Vamos fazer a mesma pergunta para diferentes "personalidades" do assistente:

In [ ]:
# Diferentes personas para testar
personas = [
    {"nome": "Professor", "system": "Você é um professor universitário que explica conceitos de forma didática e detalhada."},
    {"nome": "Amigo", "system": "Você é um amigo próximo que conversa de forma casual e descontraída."},
    {"nome": "Especialista", "system": "Você é um especialista técnico que dá respostas precisas e diretas."},
    {"nome": "Poeta", "system": "Você é um poeta que responde sempre de forma criativa e artística."}
]

pergunta = "O que você pensa sobre o futuro da tecnologia?"

print("🎭 TESTANDO DIFERENTES PERSONAS")
print("=" * 50)

for persona in personas:
    print(f"\n👤 PERSONA: {persona['nome']}")
    print("-" * 30)
    
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": persona["system"]},
            {"role": "user", "content": pergunta}
        ],
        max_completion_tokens=200,
        temperature=0.7
    )
    
    print(response.choices[0].message.content)

print("\n💡 O system message define completamente o 'jeito' do assistente!")